#### 记录一些关于地图的操作（地图与导航）

#### 1. 更改地图
    要更改地图，世界也必须随之改变。模拟将从头开始重建。您可以选择在新世界中使用相同的地图重新开始，也可以同时更改地图和世界：
    reload_world() 创建一个新的世界实例，使用相同的地图。
    load_world() 改变当前地图并创建一个新世界。

In [41]:
import carla
from numpy import random
client = carla.Client('localhost', 2000)
world = client.load_world('Town01')

# 要获取可用地图列表
print(client.get_available_maps())

['Town04_Opt', 'Town10HD', 'Town02', 'Town06_Opt', 'Town04', 'Town01', 'Town03_Opt', 'Town05_Opt', 'Town10HD_Opt', 'Town01_Opt', 'Town02_Opt', 'Town06', 'Town07_Opt', 'Town03', 'Town05', 'Town07', 'Town12', 'Town13', 'Town11', 'Town15', 'AnnotationColorLandscape']


#### 2. 地标Landmarks
    carla.Landmark 对象代表 OpenDRIVE 信号。此类的属性和方法描述了地标及其影响范围。
        carla.LandmarkOrientation 表示地标相对于道路几何定义的方位。
        carla.LandmarkType 包含常见的地标类型，以便于转换为 OpenDRIVE 类型。
    carla.Waypoint 可以获取前方一定距离内的地标。可以指定要获取的地标类型。
    carla.Map 可以检索地标集合。它可以返回地图中的所有地标，也可以返回具有相同 ID、类型或组的地标。
    carla.World 充当地标与模拟中代表它们的 carla.TrafficSign 和 carla.TrafficLight 之间的中介。

In [ ]:
my_waypoint.get_landmarks(200.0,True)

#### 3. 航点Waypoints
    每个航点都包含一个 carla.Transform，用于指示其在地图上的位置以及包含它的车道的方向。
    变量 road_id、section_id、lane_id 和 s 对应于 OpenDRIVE 道路。航点的 ID 由这四个值的哈希组合构成。
    notic：同一条道路上距离小于 2 厘米的航点共享同一个 ID。
    信息包括车道的左右车道线、用于确定其是否位于交叉路口内的布尔值、车道类型、宽度以及变道权限。

In [ ]:
# 从航点获取车道信息
inside_junction = waypoint.is_junction()
width = waypoint.lane_width
right_lm_color = waypoint.right_lane_marking.color

#### 4. 道路Lanes
    车道周围的标线信息通过 carla.LaneMarking 获取。车道标线由一系列变量定义：
    color：carla.LaneMarkingColor 是定义标线颜色的枚举值。
    lane_change：carla.LaneChange 指示车道是否允许左转、右转、左右双向或均不允许。
    type：carla.LaneMarkingType 是定义标线类型的枚举值，符合 OpenDRIVE 标准。
    width：定义标线的宽度。

In [ ]:
# 获取特定路点的车道类型、车道标线和车道变更权限信息
# 获取航点的车道类型
lane_type = waypoint.lane_type

# 确定左侧车道线的类型。
left_lanemarking_type = waypoint.left_lane_marking.type()

# 获取此航点可用的车道变更信息。
lane_change = waypoint.lane_change

#### 5. 十字路口Junctions
    carla.Junction 类包含 get_waypoints 方法，该方法返回路口内每条车道的一对路径点。每对路径点分别位于路口边界的起点和终点。

#### 6. 环境对象Environment Objects

In [ ]:
# 获取世界中的建筑物
world = client.get_world()
env_objs = world.get_environment_objects(carla.CityObjectLabel.Buildings)

# 获取各个建筑物的 ID 并保存到一组set中
building_01 = env_objs[0]
building_02 = env_objs[213]
objects_to_toggle = {building_01.id, building_02.id}

# 关闭建筑物
world.enable_environment_objects(objects_to_toggle, False)
# 开启建筑物
# world.enable_environment_objects(objects_to_toggle, True)

#### 7. CARLA导航Navigation in CARLA
    CARLA 中的导航是通过航点 API 管理的，该 API 结合了 carla.Waypoint 和 carla.Map 中的方法。
    客户端必须首先与服务器通信以检索包含航点信息的地图对象。此操作只需执行一次，所有后续查询均在客户端执行。

##### 7.1通过航点导航
    `next(d)` 创建一个位于车道方向、距离约为 d 的航点列表。该列表包含每个可能偏差对应的航点。
    `previous(d)` 创建一个位于车道反方向、距离约为 d 的航点列表。该列表包含每个可能偏差对应的航点。
    `next_until_lane_end(d)` 和 `previous_until_lane_start(d)` 返回距离为 d 的航点列表。这些列表分别从当前航点到其所在车道的终点和起点。
    `get_right_lane()` 和 `get_left_lane()` 返回相邻车道中对应的航点（如果存在）。可以通过找到右侧/左侧车道的下一个航点并移动到该航点来进行变道操作。

In [ ]:
# 找到前方2米处的下一个航点。
waypoint = waypoint.next(2.0)

##### 7.2 生成地图导航
    客户端需要向服务器发送请求以获取 .xodr 地图文件并将其解析为 carla.Map 对象

In [ ]:
# 获取地图对象：
map = world.get_map()
# 获取这些生成点的列表，每个生成点都包含一个 carla.Transform 对象
spawn_points = world.get_map().get_spawn_points()

# 行车道或人行道中心最近的航点。
waypoint01 = map.get_waypoint(vehicle.get_location(),project_to_road=True, lane_type=(carla.LaneType.Driving | carla.LaneType.Sidewalk))

# 最近的航点，但需指定 OpenDRIVE 参数。
waypoint02 = map.get_waypoint_xodr(road_id,lane_id,s)

# 生成一系列路径点来可视化城市车道。这将在地图上为每条道路和车道创建路径点。所有路径点之间的距离约为 2 米
waypoint_list = map.generate_waypoints(2.0)

# 要生成一个最小道路拓扑图，这将返回一个路径点对（元组）列表。每对路径点中的第一个元素与第二个元素相连，它们共同定义了地图中每条车道的起点和终点
waypoint_tuple_list = map.get_topology()

# 以下示例将 carla.Transform 转换为地理纬度和经度坐标，格式为 carla.GeoLocation：
my_geolocation = map.transform_to_geolocation(vehicle.transform)

# 道路信息以 OpenDRIVE 格式保存到磁盘
info_map = map.to_opendrive()

#### 8. 地图信息
    城镇概览
    Town01：一个小型、简单的城镇，有一条河流和几座桥梁。（分层地图）
    Town02：一个小型、简单的城镇，住宅和商业建筑混杂。（分层地图）
    Town03：一张较大的城市地图，有环岛和大型路口。（分层地图）
    Town04：一个嵌入群山的城镇，有一条特殊的“8”字形无限高速公路。（分层地图）
    Town05：一个方格状城镇，有十字路口和一座桥梁。每个方向都有多条车道。便于变道。（分层地图）
    Town06：一条长长的多车道高速公路，有许多高速公路入口和出口。还有一个密歇根式左转。（分层地图）
    Town07：乡村环境，道路狭窄，有玉米地、谷仓，几乎没有交通信号灯。（分层地图）
    Town08：用于排行榜挑战的秘密“未显示”城镇。
    Town09：用于排行榜挑战的秘密“未显示”城镇。
    Town10：市中心城市环境，有摩天大楼、住宅楼和海滨长廊。（分层地图）
    Town11：一张未装饰的大型地图。作为大型地图功能的概念验证。
    Town12 是一张包含众多不同区域的大型地图，其中包括高层建筑、住宅区和乡村环境。

#### 9. 分层地图的使用

In [ ]:
# 加载Town01 的分层地图，包含最小布局、建筑物和停放车辆。
world = client.load_world('Town01_Opt', carla.MapLayer.Buildings | carla.MapLayer.ParkedVehicles)

# 关闭所有建筑物
world.unload_map_layer(carla.MapLayer.Buildings)

# 打开所有建筑物
world.load_map_layer(carla.MapLayer.Buildings)